In [1]:
# Notebook setup: ensure we run from repo root (so relative paths like configs/ and runs/ work)
import os
import sys
from pathlib import Path

def _find_repo_root(start: Path | None = None) -> Path:
    p = (start or Path.cwd()).resolve()
    for cand in [p] + list(p.parents):
        if (cand / "pyproject.toml").exists() and (cand / "src").exists():
            return cand
    # Fallback: if executed from notebooks/, go one level up
    if p.name.lower() == "notebooks" and (p.parent / "src").exists():
        return p.parent
    return p

REPO_ROOT = _find_repo_root()
os.chdir(REPO_ROOT)

src_path = REPO_ROOT / "src"
if src_path.exists() and str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print("Repo root:", REPO_ROOT)

# Note:
# Set RUN_ID below to an existing run you created via the CLI (e.g., RUN_XXX_0005).


Repo root: C:\Users\Martín\Desktop\inkswarm-core\usul-inkswarm-detectlab


# 01 — Placeholder data inspection

In [3]:
from pathlib import Path
import pandas as pd

from inkswarm_detectlab.io.paths import raw_table_basepath
from inkswarm_detectlab.io.tables import read_auto

# Set this to an existing run you created via the CLI (e.g., RUN_XXX_0005)
RUN_ID = "RUN_XXX_0006"  # <-- change me
run_dir = Path("runs") / RUN_ID

def _read_required_table(event: str):
    base = raw_table_basepath(run_dir, event)
    try:
        return read_auto(base)
    except FileNotFoundError as e:
        print(f"ERROR: could not find raw parquet for event={event!r} under {run_dir}")
        if run_dir.exists():
            print("Found files under run_dir/raw/:")
            raw_dir = run_dir / "raw"
            if raw_dir.exists():
                for p in sorted(raw_dir.glob("*")):
                    print(" -", p)
            else:
                print(" - (no raw/ folder found)")
        else:
            print("Run directory does not exist:", run_dir)
            print("Tip: make sure you launched Jupyter from the repo root, or use the setup cell above.")
        raise

login = _read_required_table("login_attempt")
checkout = _read_required_table("checkout_attempt")

print("login shape:", login.shape)
display(login.head())

print("checkout shape:", checkout.shape)
display(checkout.head())


login shape: (10802, 27)


,run_id,event_id,event_ts,user_id,session_id,ip_hash,device_fingerprint_hash,country,is_fraud,label_replicators,...,mfa_used,mfa_result,support_contacted,support_channel,support_responder_type,support_wait_seconds,support_handle_seconds,support_cost_usd,support_resolution,support_offset_seconds
0,RUN_XXX_0006,login_0000000021,2025-12-01 00:05:48-03:00,user_00028,sess_29751,ip_0600,dev_9462,AR,False,False,...,False,not_applicable,False,none,none,NaN,NaN,NaN,none,NaN
1,RUN_XXX_0006,login_0000000024,2025-12-01 00:08:05-03:00,user_00024,sess_57317,ip_9179,dev_7941,AR,False,False,...,True,fail,False,none,none,NaN,NaN,NaN,none,NaN
2,RUN_XXX_0006,login_0000000023,2025-12-01 00:09:08-03:00,user_00090,sess_53322,ip_6508,dev_4884,AR,False,False,...,False,not_applicable,False,none,none,NaN,NaN,NaN,none,NaN
3,RUN_XXX_0006,login_0000000001,2025-12-01 00:09:15-03:00,user_00123,sess_55187,ip_0124,dev_2831,AR,False,False,...,True,pass,False,none,none,NaN,NaN,NaN,none,NaN
4,RUN_XXX_0006,login_0000000014,2025-12-01 00:13:16-03:00,user_00072,sess_85335,ip_5698,dev_0411,AR,False,False,...,False,not_applicable,False,none,none,NaN,NaN,NaN,none,NaN


checkout shape: (1106, 17)


,run_id,event_id,event_ts,user_id,session_id,ip_hash,device_fingerprint_hash,country,is_fraud,metadata_json,payment_value,basket_size,is_first_time_user,is_premium_user,credit_card_hash,checkout_result,decline_reason
0,RUN_XXX_0006,checkout_0000000002,2025-12-01 00:04:51-03:00,user_00090,sess_53322,ip_6508,dev_4884,AR,False,"{""campaign_id"":null,""note"":""fraud labels disab...",6.319103,6,False,True,cc_064817,success,None
1,RUN_XXX_0006,checkout_0000000001,2025-12-01 00:46:37-03:00,user_00101,sess_19375,ip_8295,dev_0585,AR,False,"{""campaign_id"":null,""note"":""fraud labels disab...",23.471192,3,False,False,cc_044687,success,None
2,RUN_XXX_0006,checkout_0000000003,2025-12-01 02:22:08-03:00,user_00112,sess_59100,ip_2238,dev_6179,AR,False,"{""campaign_id"":null,""note"":""fraud labels disab...",47.120640,2,False,True,cc_042634,success,None
3,RUN_XXX_0006,checkout_0000000006,2025-12-01 03:10:33-03:00,user_00014,sess_58105,ip_1615,dev_4963,AR,False,"{""campaign_id"":null,""note"":""fraud labels disab...",31.262673,6,False,False,cc_117133,success,None
4,RUN_XXX_0006,checkout_0000000004,2025-12-01 03:28:11-03:00,user_00003,sess_06375,ip_4713,dev_0149,AR,False,"{""campaign_id"":null,""note"":""fraud labels disab...",104.954351,1,True,False,cc_014127,success,None
